In [ ]:
!pip install ray

     |████████████████████████████████| 48.5MB 94kB/s 
     |████████████████████████████████| 204kB 46.8MB/s 
     |████████████████████████████████| 133kB 30.8MB/s 
     |████████████████████████████████| 2.9MB 40.2MB/s 
     |████████████████████████████████| 81kB 10.3MB/s 
     |████████████████████████████████| 1.3MB 48.9MB/s 
     |████████████████████████████████| 71kB 4.2MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 143kB 40.8MB/s 
     |████████████████████████████████| 296kB 43.4MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp36-none-any.whl size=12622 sha256=118b6257bd5be417b0b4c35aa20ff70fabdee4ad8e454df5a6be8abb669bfb55
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl siz

In [ ]:
import ray
import time
ray.init()

2021-01-12 01:54:06,691	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


{'metrics_export_port': 61951,
 'node_id': '279b22a480d432e3457cd2891c98be5f58663cc8',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-01-12_01-54-06_044642_57/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-01-12_01-54-06_044642_57/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-01-12_01-54-06_044642_57',
 'webui_url': '127.0.0.1:8265'}

## ray

remote function은 Ray의 프로세스에 의해 비동기적으로 실행됩니다. 따라서 아래의 코드는 ray에 의해서 비동기적으로 실행됩니다.

In [ ]:
@ray.remote(num_cpus=0.5)
def f(x):
    time.sleep(1)
    return x

In [ ]:
start = time.time()
# Start 4 tasks in parallel.
result_ids = []
for i in range(4):
    result_ids.append(f.remote(i))

# Wait for the tasks to complete and retrieve the results.
# With at least 4 cores, this will take 1 second.
results = ray.get(result_ids)  # [0, 1, 2, 3]

print("Ray 소요시간: {}".format(time.time() - start))

Ray 소요시간: 2.018064260482788


In [ ]:
def f(x):
    time.sleep(1)
    return x

In [ ]:
start = time.time()
result_ids = [] 
for i in range(4):
    result_ids.append(f(i))
print("소요시간 : {}".format(time.time() - start))

소요시간 : 4.003791570663452


## Task in Task

병렬로 x_id와 y_id를 연산하고 z_id를 구하는 예제

In [ ]:
import numpy as np

@ray.remote
def create_matrix(size):
    return np.random.normal(size=size)

@ray.remote
def multiply_matrices(x, y):
    return np.dot(x, y)

size = 10000

# Get the results with ray.
start = time.time()
x_id = create_matrix.remote([size, size])
y_id = create_matrix.remote([size, size])
z_id = multiply_matrices.remote(x_id, y_id)
z = ray.get(z_id)
print("z_id_with_ray : {}".format(time.time() - start))

# Get the results with np array.
start = time.time()
x_id = np.random.normal([size, size])
y_id = np.random.normal([size, size])
z_id = np.dot(x_id, y_id)
print("z_id_with_np_array : {}".format(time.time() - start))

2021-01-12 02:20:42,049	WARNING worker.py:1034 -- A worker died or was killed while executing task fafba2bafaed5dc3ffffffff01000000.
2021-01-12 02:21:17,836	WARNING worker.py:1034 -- A worker died or was killed while executing task b7603b6cc81d46c1ffffffff01000000.
2021-01-12 02:21:25,181	ERROR worker.py:980 -- Possible unhandled error from worker: ray::multiply_matrices() (pid=808, ip=172.28.0.2)
  File "python/ray/_raylet.pyx", line 425, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 447, in ray._raylet.execute_task
ray.exceptions.WorkerCrashedError: The worker died unexpectedly while executing this task.


z_id_with_ray : 129.8542284965515
z_id_with_np_array : 0.011426448822021484


## Aggregation

연산을 어떻게 병렬로 배치하느냐에 따라서 보는 속도의 차이
![](https://miro.medium.com/max/1400/1*vHz3troEmr4uLns0V8VmdA.jpeg)


In [ ]:
import time

@ray.remote
def add(x, y):
    time.sleep(1)
    return x + y

In [ ]:
start = time.time()
# Aggregate the values slowly. This approach takes O(n) where n is the
# number of values being aggregated. In this case, 7 seconds.
id1 = add.remote(1, 2)
id2 = add.remote(id1, 3)
id3 = add.remote(id2, 4)
id4 = add.remote(id3, 5)
id5 = add.remote(id4, 6)
id6 = add.remote(id5, 7)
id7 = add.remote(id6, 8)
result = ray.get(id7)
print("Vanilla version : {}".format(time.time() - start))


start = time.time()
# Aggregate the values in a tree-structured pattern. This approach
# takes O(log(n)). In this case, 3 seconds.
id1 = add.remote(1, 2)
id2 = add.remote(3, 4)
id3 = add.remote(5, 6)
id4 = add.remote(7, 8)
id5 = add.remote(id1, id2)
id6 = add.remote(id3, id4)
id7 = add.remote(id5, id6)
result = ray.get(id7)
print("Advanced version : {}".format(time.time() - start))

Vanilla version : 7.023898363113403
Advanced version : 4.0150158405303955


## Reference

https://towardsdatascience.com/modern-parallel-and-distributed-python-a-quick-tutorial-on-ray-99f8d70369b8